In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
# Cargamos la base de Datos Iris 
iris = load_iris()
X = iris.data
y = iris.target

# Codificamos las etiquetas como one-hot, es decir, creamos una matriz 
# de ceros y unos, donde los 1's se agregan en la columna con el mismo valor de
#la etiqueta y las demas entradas son cero.
y_onehot = np.zeros((len(y), 3))
y_onehot[np.arange(len(y)), y] = 1
# Separamos los datos de entrenamiento y de prueba considerando que ahora 
# que las etiquetas ahora se toman de la matriz one hot
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2)

#Separamos el tamaño del input, damos el numero de capaz ocultas y el tamaño de 
#salida.
input_size = X_train.shape[1]
hidden_size = 50
output_size = 3

# Inicializamos los pesos de manera random para la capa oculta y la capa de sali
# da. Además inicializamos los baias como un vector de ceros.
W1 = .5*np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = .5*np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

# Definimos las funciones de activación y la función de perdida 
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Definimos la función de pérdida (Entropía Cruzada)
def binary_cross_entropy(y_pred, y_true):
    epsilon = 1e-12
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)  # Clippear los valores para evitar log(0)
    n = y_pred.shape[0]
    loss = -(1/n) * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss



In [ ]:
#Definimos la función de predicción 
def pred(X,W1,b1,W2,b2):
  hidden_layer = np.dot(X, W1) + b1
  hidden_layer_act = sigmoid(hidden_layer)
  ouput_layer = np.dot(hidden_layer_act, W2) + b2
  output_layer_act = softmax(ouput_layer)
  return output_layer_act
# Implementamos el algoritmo descenso Gradiente Descendiente Estocástico
def stochastic_gradient_descent(X, y, W1, b1, W2, b2, learning_rate):
    num_samples = X.shape[0]
    for i in range(num_samples):
        # Implementamos la propagación hacia delante 
        hidden_layer = np.dot(X, W1) + b1
        hidden_layer_act = sigmoid(hidden_layer)
        ouput_layer = np.dot(hidden_layer_act, W2) + b2
        output_layer_act = softmax(ouput_layer)
        # Calculamos la pérdida y el gradiente con la Entropía Cruzada
        loss = binary_cross_entropy(output_layer_act, y)
        Grad_Out_La = output_layer_act - y
        Grad_W2 = np.dot(hidden_layer_act.T, Grad_Out_La)
        Grad_b2 = np.sum(Grad_Out_La, axis= 0)
        Grad_Out_La_Act = np.dot(Grad_Out_La, W2.T)
        Grad_Hid_La = Grad_Out_La_Act * (hidden_layer > 0)
        Grad_W1 = np.dot(X.T, Grad_Hid_La)
        Grad_b1 = np.sum(Grad_Hid_La, axis= 0)
        # Propagamos hacia atrás para actualizar los pesos y los sesgos
        W1 -= learning_rate * Grad_W1
        b1 -= learning_rate * Grad_b1
        W2 -= learning_rate * Grad_W2 
        b2 -= learning_rate * Grad_b2
    return W1, b1, W2, b2


In [ ]:
learning_rate= 0.1
num_epochs= 300
batch_size= 40

for epoch in range(num_epochs):
    # Randomizamos los datos de entrenamiento
    indices = np.random.permutation(X_train.shape[0])
    X_train = X_train[indices]
    y_train = y_train[indices]

    # Dividimos  los datos de entrenamiento en lotes
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        W1, b1, W2, b2 = stochastic_gradient_descent(X_batch, y_batch, W1, b1, W2, b2, learning_rate)
    y_pred= pred(X_train,W1,b1,W2,b2)
    y_pred_ = np.argmax(y_pred, axis=1)
    y_train_ = np.argmax(y_train, axis=1)
    print("Accuracy en la epoca ", epoch, " : ", accuracy_score(y_train_, y_pred_))

Accuracy en la epoca  0  :  0.975
Accuracy en la epoca  1  :  0.975
Accuracy en la epoca  2  :  0.975
Accuracy en la epoca  3  :  0.975
Accuracy en la epoca  4  :  0.9833333333333333
Accuracy en la epoca  5  :  0.975
Accuracy en la epoca  6  :  0.9833333333333333
Accuracy en la epoca  7  :  0.9666666666666667
Accuracy en la epoca  8  :  0.975
Accuracy en la epoca  9  :  0.9666666666666667
Accuracy en la epoca  10  :  0.975
Accuracy en la epoca  11  :  0.9666666666666667
Accuracy en la epoca  12  :  0.9583333333333334
Accuracy en la epoca  13  :  0.9666666666666667
Accuracy en la epoca  14  :  0.9666666666666667
Accuracy en la epoca  15  :  0.9666666666666667
Accuracy en la epoca  16  :  0.975
Accuracy en la epoca  17  :  0.975
Accuracy en la epoca  18  :  0.975
Accuracy en la epoca  19  :  0.975
Accuracy en la epoca  20  :  0.975
Accuracy en la epoca  21  :  0.975
Accuracy en la epoca  22  :  0.975
Accuracy en la epoca  23  :  0.9666666666666667
Accuracy en la epoca  24  :  0.983333333

In [ ]:
y_pred

array([[1.10531386e-03, 9.98894682e-01, 3.75408491e-09],
       [9.79903331e-01, 2.00966694e-02, 5.92029825e-22],
       [3.42013525e-07, 1.07232323e-02, 9.89276426e-01],
       [9.99941869e-01, 5.81308037e-05, 9.36657082e-23],
       [2.60871307e-09, 3.19123744e-05, 9.99968085e-01],
       [9.96011219e-01, 3.98878139e-03, 1.69990354e-18],
       [9.99973868e-01, 2.61323893e-05, 2.34718743e-22],
       [9.99907780e-01, 9.22195877e-05, 3.79959339e-23],
       [1.61163978e-12, 5.69876424e-08, 9.99999943e-01],
       [1.08176544e-07, 9.84370607e-03, 9.90156186e-01],
       [9.99890525e-01, 1.09475184e-04, 3.71920176e-22],
       [9.99858160e-01, 1.41839769e-04, 9.62985294e-20],
       [2.60591124e-04, 9.99735852e-01, 3.55640982e-06],
       [9.99969769e-01, 3.02309473e-05, 3.03180962e-22],
       [1.42490898e-09, 5.52192386e-05, 9.99944779e-01],
       [7.27651067e-07, 2.35461646e-02, 9.76453108e-01],
       [9.99998644e-01, 1.35569552e-06, 7.73485953e-24],
       [9.99990455e-01, 9.54502

In [ ]:
#Metemos los datos de prueba en la red ya entrenada 
y_pred= pred(X_test,W1,b1,W2,b2)
#Los devolvemos a sus etiquetas originales 
y_pred_ = np.argmax(y_pred, axis=1)
y_test_ = np.argmax(y_test, axis=1)
#Calculamos la precisión del modelo con los datos que reservamos que no fueron 
#usados para el entrenamiento
print("Accuracy: ", accuracy_score(y_test_, y_pred_))

Accuracy:  0.9666666666666667


En conclusión...
Notemos que el algoritmo propaga el error a través de las capas; desde la capa de salida hasta la capa de entrada, ajusta los pesos y los bias en función del error. 

Así, a diferencia de la regresión lógista, podemos hacer que la red aprenda tareas especificas, en este caso, con los datos utilizados podemos clasificar los distintos tipos de flores con las etiquetas dadas. Sin embargo, también es posible, con las modificaciones adecuadas, la predicción de valores numéricos en cualquier otro contexto. Y muchos otros ejemplos...